### Objective
Create San Diego Block group boundaries along with business variables. This would be achieved by using an enriched file we had create at block level.

In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection, GeoAccessor
import sys
sys.path.append('../')
sys.path.append('../../')
from utils import get_config, format_json
import pandas as pd
from arcgis.geometry  import Geometry
import json
from feature_layer_utils import get_enrichment_variables, create_feature_layer
from itertools import chain

In [2]:
from arcgis.features.geo._array import GeoArray

In [3]:
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=get_config("arcgis","clientid"))

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=v5kf7vosEPb7qo2I9LKkz6LYOGaIBW&allow_verification=false
Enter code obtained on signing in using SAML: ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
df = pd.read_csv("../resources/full_enriched_san_diego_county_block_groups_cleaned.csv")
print(f"Shape: {df.shape}")
df.head(3)

Shape: (2057, 4241)


,objectid,state_abbr,state_fips,county_fips,stcofips,tract_fips,blockgroup_fips,fips,pop_sqmi,sqmi,...,x1130fy_a,x1130fy_i,x1002fy_x,x1002fy_a,x1002fy_i,x1003fy_x,x1003fy_a,x1003fy_i,SHAPE,totpop_cy
0,29002,CA,6,73,6073,100,1,60730001001,4788.0,0.25,...,11293.00,225.0,12946134.0,26693.06,218.0,7469031.0,15400.06,214.0,"{'rings': [[[-117.18894090906399, 32.758807764...",1199.0
1,29003,CA,6,73,6073,100,2,60730001002,5032.4,0.34,...,11843.34,236.0,19996145.0,28484.54,233.0,11682118.0,16641.19,231.0,"{'rings': [[[-117.18715990828298, 32.756859764...",1692.0
2,29004,CA,6,73,6073,201,1,60730002011,4385.0,0.20,...,7167.84,143.0,7516044.0,17120.83,140.0,4369363.0,9952.99,138.0,"{'rings': [[[-117.168410904985, 32.75683776569...",902.0


Since we are reading from CSV file , the FIPS having a leading '0' are getting dropped. I am just adding them back.

In [5]:
df['fips'] = df['fips'].astype(str)
df['fips'] = '0' + df['fips']
print(f"Shape: {df.shape}")
df.head(3)

Shape: (2057, 4241)


,objectid,state_abbr,state_fips,county_fips,stcofips,tract_fips,blockgroup_fips,fips,pop_sqmi,sqmi,...,x1130fy_a,x1130fy_i,x1002fy_x,x1002fy_a,x1002fy_i,x1003fy_x,x1003fy_a,x1003fy_i,SHAPE,totpop_cy
0,29002,CA,6,73,6073,100,1,060730001001,4788.0,0.25,...,11293.00,225.0,12946134.0,26693.06,218.0,7469031.0,15400.06,214.0,"{'rings': [[[-117.18894090906399, 32.758807764...",1199.0
1,29003,CA,6,73,6073,100,2,060730001002,5032.4,0.34,...,11843.34,236.0,19996145.0,28484.54,233.0,11682118.0,16641.19,231.0,"{'rings': [[[-117.18715990828298, 32.756859764...",1692.0
2,29004,CA,6,73,6073,201,1,060730002011,4385.0,0.20,...,7167.84,143.0,7516044.0,17120.83,140.0,4369363.0,9952.99,138.0,"{'rings': [[[-117.168410904985, 32.75683776569...",902.0


In [6]:
general_columns_retained = ["fips", "SHAPE"]


In [7]:
data_variables = get_enrichment_variables('business')

Parsing [Esri Business Data] for business segment!!
	Number of Variables: 99


Observations:  
* Looks like all the variables were not harvested in out enrichment data.
* We would pick the interection of varibales from our list and the enriched data we have.

In [8]:
intersection_list = list(set(df.columns) & set(data_variables))

In [9]:
print(f"Size of Variables extracted: {len(data_variables)}")
print(f"Size of Variables from enriched data: {len(df.columns)}") # this could have additional columsn we dont care about.
print(f"Size of intersection: {len(intersection_list)}")

Size of Variables extracted: 99
Size of Variables from enriched data: 4241
Size of intersection: 99


In [10]:
intersection_list = list(chain(general_columns_retained, intersection_list))

There is no documentation but it seems, I am not able to publish feature layer with more than 990 fields. So We would equally divide the variable list for respective segment.

In [11]:
%%time
feature_layer = create_feature_layer(df, 
                                     gis=gis,
                                     final_columns=intersection_list,
                                     title="San Diego Block Groups with Business Variables Data",
                                     tags=['SanDiego', 'Business'],
                                    )

/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/arc_gis/feature_layers/feature_layer_utils.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SHAPE"] = GeoArray([Geometry(json.loads(format_json(x))) for x in df['SHAPE']])


Number of fields published as part of this layer: 103


<Item title:"San Diego Block Groups with Business Variables Data" type:Feature Layer Collection owner:akale_UCSDOnline>

CPU times: user 5.01 s, sys: 177 ms, total: 5.19 s
Wall time: 26 s
